In [1]:
import pandas as pd
import numpy as np
from TextHandler import *

In [2]:
df_1= pd.read_csv('data/arabic_categorization_data.csv')
df_1.drop(df_1.columns[0], axis=1, inplace=True)
df_1.dropna(inplace=True)
print(df_1.columns)

df_2= pd.read_csv('data/processed_data.csv')
df_2.drop(df_2.columns[0], axis=1, inplace=True)
df_2.dropna(inplace=True)
print(df_2.columns)

Index(['text', 'type'], dtype='object')
Index(['Content', 'Link', 'Category'], dtype='object')


In [3]:
data=df_1['text']
print(data[:1])
preprocessed_data_1=preprocess_text(data)

0    \r\nأشرف رئيس الجمهورية الباجي قايد السبسي الي...
Name: text, dtype: object


In [4]:
data=df_2['Content']
print(data[:1])
preprocessed_data_2=preprocess_text(data)


C:\Users\Amr\AppData\Local\Temp\ipykernel_21888\3943842632.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(data[:1])


0    جاءت جوله التمويل التمهيديه الاولي للشركه بقيا...
Name: Content, dtype: object


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(preprocessed_data_1, df_1['type'], test_size=0.2, random_state=42)

# ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

# classifiers = [
#     LogisticRegression(max_iter=1000),
#     MultinomialNB(),
#     KNeighborsClassifier(),
#     RandomForestClassifier(),
#     SVC()
# ]

# for classifier in classifiers:
#     classifier_name = classifier.__class__.__name__
#     print(f"Classifier: {classifier_name}")
#     for ngram_range in ngram_ranges:
#         vectorizer = TfidfVectorizer(ngram_range=ngram_range)
#         X_train_tfidf = vectorizer.fit_transform(X_train)
#         X_test_tfidf = vectorizer.transform(X_test)

#         classifier.fit(X_train_tfidf, y_train)

#         y_pred = classifier.predict(X_test_tfidf)

#         accuracy = accuracy_score(y_test, y_pred)
#         print(f"Accuracy for n-gram range {ngram_range}: {accuracy}")


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data_2, df_2['Category'], test_size=0.2, random_state=42)

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(max_iter=1000),
    MultinomialNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(),
    SVC()
]

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    print(f"Classifier: {classifier_name}")
    for ngram_range in ngram_ranges:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range)
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        classifier.fit(X_train_tfidf, y_train)

        y_pred = classifier.predict(X_test_tfidf)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for n-gram range {ngram_range}: {accuracy}")


Classifier: LogisticRegression
Accuracy for n-gram range (1, 1): 0.9761620977353993
Accuracy for n-gram range (2, 2): 0.9725864123957092
Accuracy for n-gram range (3, 3): 0.9618593563766389
Accuracy for n-gram range (4, 4): 0.9249106078665077
Classifier: MultinomialNB
Accuracy for n-gram range (1, 1): 0.9493444576877235
Accuracy for n-gram range (2, 2): 0.9576877234803337
Accuracy for n-gram range (3, 3): 0.9558998808104887
Accuracy for n-gram range (4, 4): 0.9016686531585221
Classifier: KNeighborsClassifier
Accuracy for n-gram range (1, 1): 0.9415971394517283
Accuracy for n-gram range (2, 2): 0.964243146603099
Accuracy for n-gram range (3, 3): 0.9529201430274136
Accuracy for n-gram range (4, 4): 0.8224076281287247
Classifier: RandomForestClassifier
Accuracy for n-gram range (1, 1): 0.9630512514898689
Accuracy for n-gram range (2, 2): 0.9600715137067938
Accuracy for n-gram range (3, 3): 0.9135876042908224
Accuracy for n-gram range (4, 4): 0.799165673420739
Classifier: SVC
Accuracy for 

In [18]:
import torch
import scipy.sparse
from torch import nn
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data_2, df_2['Category'], test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


# turn X_train_tfidf to float32 to not take too much memory
X_train_tfidf = X_train_tfidf.astype(np.float32)
X_test_tfidf = X_test_tfidf.astype(np.float32)

# use half the elements of the sparse matrix
X_train_tfidf = scipy.sparse.csr_matrix(X_train_tfidf[:int(X_train_tfidf.shape[0] / 3)])
X_test_tfidf = scipy.sparse.csr_matrix(X_test_tfidf[:int(X_test_tfidf.shape[0] / 3)])
y_train = y_train[:int(y_train.shape[0] / 3)]
y_test = y_test[:int(y_test.shape[0] / 3)]

# check all unique text values in y_train and y_test, and assign numbers for each unique value in y_train and y_test, create a dictionary for this, then replace the values in y_train and y_test with the numbers, make sure to store the dictionary in a variable
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)
y_train['Category'] = y_train['Category'].astype('category')
y_test['Category'] = y_test['Category'].astype('category')
y_train_dict = dict(enumerate(y_train['Category'].cat.categories))
y_test_dict = dict(enumerate(y_test['Category'].cat.categories))
y_train['Category'] = y_train['Category'].cat.codes
y_test['Category'] = y_test['Category'].cat.codes

# convert to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()





print(y_train)
# convert to torch tensors
X_train_torch = torch.from_numpy(X_train_tfidf.todense())
X_test_torch = torch.from_numpy(X_test_tfidf.todense())
#convert y to torch tensor (y is text)
y_train_torch = torch.from_numpy(y_train)
y_test_torch = torch.from_numpy(y_test)


model = nn.Sequential(
    nn.Linear(X_train_torch.shape[1], 64),
    nn.ReLU(),
    nn.Linear(64, df_2['Category'].nunique()),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()
logps = model(X_train_torch)

loss = criterion(logps, y_train_torch)
loss.backward()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

epochs = 50
for e in range(epochs):
    optimizer.zero_grad()
    output = model.forward(X_train_torch)
    loss = criterion(output, y_train_torch)
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        model.eval()
        log_ps = model(X_test_torch)
        test_loss = criterion(log_ps, y_test_torch)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y_test_torch.view(*top_class.shape)
        test_accuracy = torch.mean(equals.float())



C:\Users\Amr\AppData\Local\Temp\ipykernel_21888\952271838.py:23: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y_train[:int(y_train.shape[0] / 3)]
C:\Users\Amr\AppData\Local\Temp\ipykernel_21888\952271838.py:24: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y_test[:int(y_test.shape[0] / 3)]


[[2]
 [1]
 [1]
 ...
 [2]
 [2]
 [0]]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported